#  Mnist認識CNN-LSTM

In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape, LSTM
from keras.callbacks import EarlyStopping

#GPUの仕様に関する設定
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(
        visible_device_list = "1",
        allow_growth = True,
        per_process_gpu_memory_fraction = 0.3))
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


## Mnist読み込み

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0
y_train = np_utils.to_categorical(y_train, num_classes = 10)

## モデルの作成

In [3]:
model = Sequential([
    Reshape((28,28,1), input_shape=(28,28)),
    Conv2D(10, (5,5), activation = 'relu'),
    Conv2D(5, (3,3), activation = 'relu'),
    MaxPooling2D(pool_size=(2,2)),
    Reshape((5,121)),
    LSTM(50, kernel_initializer = 'he_normal', return_sequences=True, activation = 'relu'),
    Dropout(0.2),
    Reshape((250,),input_shape=(5,50)),
    Dense(10, activation = 'softmax')
])
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 10)        260       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 5)         455       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 5)         0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 5, 121)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 50)             34400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 50)             0         
__________

In [4]:
es = EarlyStopping(monitor='val_loss')
model.fit(x_train, y_train, batch_size = 1000, validation_split=0.2, nb_epoch=300, callbacks=[es])

/usr/local/src/pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 48000 samples, validate on 12000 samples
Epoch 1/300
48000/48000 [==============================] - 3s 59us/step - loss: 1.6310 - acc: 0.4778 - val_loss: 0.4764 - val_acc: 0.8692
Epoch 2/300
48000/48000 [==============================] - 1s 28us/step - loss: 0.4244 - acc: 0.8686 - val_loss: 0.2323 - val_acc: 0.9341
Epoch 3/300
48000/48000 [==============================] - 1s 28us/step - loss: 0.2516 - acc: 0.9251 - val_loss: 0.1656 - val_acc: 0.9546
Epoch 4/300
48000/48000 [==============================] - 1s 27us/step - loss: 0.1886 - acc: 0.9424 - val_loss: 0.1316 - val_acc: 0.9646
Epoch 5/300
48000/48000 [==============================] - 1s 28us/step - loss: 0.1527 - acc: 0.9540 - val_loss: 0.1124 - val_acc: 0.9691
Epoch 6/300
48000/48000 [==============================] - 1s 27us/step - loss: 0.1259 - acc: 0.9610 - val_loss: 0.0948 - val_acc: 0.9747
Epoch 7/300
48000/48000 [==============================] - 1s 28us/step - loss: 0.1077 - acc: 0.9678 - val_loss: 0.0915 - 

In [5]:
predict = model.predict_classes(x_test)
print(sum(predict == y_test)/ 10000.0)

0.9832
